## Face Recognition - BCSAI Pablo Ortega

Explanation of the library:

# Training and Inference

Firstly, the we have to set the ground truth of the problem, which will be a dictionary of the people from the class with a number assigned to each of them. The ground_truth that I uses is based on the one from the labels of Rayane.

In [11]:
import cv2
import dlib
from simple_facerec import SimpleFacerec
from sklearn.metrics import f1_score
import csv
import pandas as pd

In [1]:
ground_truth = { 
"No face":0,
"Unknown":1 ,
"Alejandro_Galvez":2 ,
"Alejandro_Perez":3 ,
"Antonio_Fernandez":4,
"Bartosz_Rzycki":5 ,
"Beltran_Valle":6,
"Boumediene_Rayane":7 ,
"Carlo_Pastor":8 ,
"Carlos_Tapia":9,
"Diego_SanMartin":10 ,
"Dyanna_Monique_Rivera":11 ,
"Farah_Abuatiyeh":12, 
"Francisco_Heshiki":13,
"Henning_Gruhl":14 ,
"Ignacio_Chirinos":15 ,
"Ismael_Doukkali":16,
"Joaquin_Casasus":17,
"Juan_Jose_Rubiales":18,
"Lorenz_Ehrlich":19,
"Manuel_SanRomanGarcia":20,
"Marta_Garcia":21,
"Mikhaile_Collins":22,
"Pablo_Ortega":23,
"Rabindra_Adhikari":24,
"Tamar_Alphaidze":25,
"Tomas_Vintimilla":26,
"Vera_Prohaska":27,
"Yahya_Laraqui":28,
"Yousif_Alsaffar":29}

In [2]:
#change the format of the dictionary, so that the key is the number and the value is the name
ground_truth = {v: k for k, v in ground_truth.items()}
ground_truth[0] = "Unknown"

I created a function so I can take the labels from the test dataset given by Rayane, Yahya and Francisco.

In [8]:
def parse_yolo_format(file_path):
    '''
    This function parses the yolo format and returns the ground truth label

    Input:
    file_path: path to the yolo file

    Output:
    ground_truth: ground truth label

    '''
    with open(file_path, 'r') as file:
        line = file.readlines()
        label = line[0].split(' ')[0]
        bbox = line[0].split(' ')[1:]
        bbox = [float(i) for i in bbox]
        

    return ground_truth[int(label)]

#Example with the 210th frame of the video of Yahya
yolo_file_path = 'labels/labelled_frames_Video_Yahya_Laraqui/210.txt'
parsed_data = parse_yolo_format(yolo_file_path)
print(parsed_data)

Unknown


Now I will create a function that will take the labels from the test dataset from each of the authors. The function will take the labels from the test dataset and will return a list of the labels.

In [9]:
dicc_Rayane = {"author" : "Rayane","label_link" : 'labels/labelled_frames_Video_Boumediene_Rayane/' , "n_labels" : 5856 }
dicc_Yahya = {"author" : "Yahya", "label_link" : 'labels/labelled_frames_Video_Yahya_Laraqui/' , "n_labels" : 5227 }
dicc_Francisco = {"author": "Francisco", "label_link" : 'labels/labelled_frames_Video_Francisco_Heshiki/Video_Francisco_Heshiki_0000' , "n_labels" : 6457 }
def takeLabels(dicc):
    '''
    This function takes the labels of the frames of a video and returns a list with the labels
    
    Input:
    dicc: dictionary with the author of the video, the link to the labels and the number of labels
    
    Output:
    labels: list with the labels of the frames of the video
    '''
    
    labels = []
    for i in range(1, dicc["n_labels"]+1):
        if dicc["author"] == "Francisco":
            yolo_file_path = dicc["label_link"] + str(i).zfill(4)+'.txt'
        else:
            yolo_file_path = dicc["label_link"] +str(i)+'.txt'
        parsed_data = parse_yolo_format(yolo_file_path)
        labels.append(parsed_data)
    return labels

labelsRayane = takeLabels(dicc_Rayane)
labelsYahya = takeLabels(dicc_Yahya)
#labelsFrancisco = takeLabels(dicc_Francisco)


I could not take the labels from the video of Francisco because there are some frames that are not labelled, so i got an error when running the function takeLabels(dicc_Francisco)

In [10]:
len(labelsRayane), len(labelsYahya)#, len(labelsFrancisco)

(5856, 5227)

I take the predicted labels from each of the videos using the model.

In [17]:
#predicted labels
predicted_labels = []

#videos
video1="videos/Video_Boumediene_Rayane.mp4"
video2="videos/Video_Yahya_Laraqui.mp4"
video3="videos/Video_Francisco_Heshiki.mp4" 

#load camera
cap= cv2.VideoCapture(video3)

#encode faces from a folder
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

while True:
    ret, frame = cap.read()
    name = "Unknown"
    #detect faces
    face_locations, face_names = sfr.detect_known_faces(frame)
    for face_loc, name in zip(face_locations, face_names):
        top, right, bottom, left = face_loc[0], face_loc[1], face_loc[2], face_loc[3]


        # print a square in the middle of the window
        cv2.rectangle(frame, (70, 380), (570, 470), (133, 228, 0), -1)
        # put some text in the middle of the square
        cv2.putText(frame, name.replace("_", " "), (230, 430), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)


    if name == "Unknown":

        # print a square in the middle of the window
        cv2.rectangle(frame, (70, 380), (570, 470), (0, 100, 255), -1)
        # put some text in the middle of the square
        cv2.putText(frame, name, (230, 430), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        # append the name to the list of predicted labels
        
    print(name)
    predicted_labels.append(name)


    #cv2.imshow("frame", frame)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()




28 encoding images found.
Encoding images loaded
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Unknown
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya_Laraqui
Yahya

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


I save them in a csv so I dont have to run the model again.

In [20]:
#save predicted labels in new a csv file
with open('predicted_labels_Francisco.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Label"])
    for i in predicted_labels:
        writer.writerow([i])

# Analysis of the results

I will check the performance of the model using the F1 score.

In [22]:
def calculate_f1_score( y_true, y_pred, name):

    f1 = f1_score(y_true, y_pred, average='weighted')

    #Analyze the results
    print("F1 score: ", f1, ", with the video of", name)

In [23]:
#rayane
rayane_predicted_labels = pd.read_csv("predicted_labels_Rayane.csv")["Label"].tolist()
calculate_f1_score(labelsRayane, rayane_predicted_labels, "Rayane")


F1 score:  0.7973330171516066 , with the video of Rayane


In [24]:
#yahya
yahya_predicted_labels = pd.read_csv("predicted_labels_Yahya.csv")["Label"].tolist()
calculate_f1_score(labelsYahya, yahya_predicted_labels, "Yahya")


F1 score:  0.7186339528421226 , with the video of Yahya
